## Data Generation

In [30]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from myPackage.my_module import *
from torchvision.transforms import ToTensor

In [5]:
def projection(matrix):
    return np.real(np.matrix([[matrix[0,0],0,0,matrix[0,3]/2+matrix[3,0]/2],\
                                [0,matrix[2,2]/2+matrix[1,1]/2,0,0],\
                                [0,0,matrix[2,2]/2+matrix[1,1]/2,0],\
                                [matrix[0,3]/2+matrix[3,0]/2,0,0,matrix[3,3]]]))
samples = []
for i in range(10000):
    initial_matrix = rand_PSDM(4)
    projected = projection(initial_matrix)
    parameters = [projected[0,0], projected[0,3], projected[1,1], projected[3,3]]
    projected = density_matrix(projected)
    projected.set(20000)
    samples.append((projected.data, parameters))
    if (i+1)%10 == 0:
        print(f'{i+1}/10000 done')
    

10/10000 done
20/10000 done
30/10000 done
40/10000 done
50/10000 done
60/10000 done
70/10000 done
80/10000 done
90/10000 done
100/10000 done
110/10000 done
120/10000 done
130/10000 done
140/10000 done
150/10000 done
160/10000 done
170/10000 done
180/10000 done
190/10000 done
200/10000 done
210/10000 done
220/10000 done
230/10000 done
240/10000 done
250/10000 done
260/10000 done
270/10000 done
280/10000 done
290/10000 done
300/10000 done
310/10000 done
320/10000 done
330/10000 done
340/10000 done
350/10000 done
360/10000 done
370/10000 done
380/10000 done
390/10000 done
400/10000 done
410/10000 done
420/10000 done
430/10000 done
440/10000 done
450/10000 done
460/10000 done
470/10000 done
480/10000 done
490/10000 done
500/10000 done
510/10000 done
520/10000 done
530/10000 done
540/10000 done
550/10000 done
560/10000 done
570/10000 done
580/10000 done
590/10000 done
600/10000 done
610/10000 done
620/10000 done
630/10000 done
640/10000 done
650/10000 done
660/10000 done
670/10000 done
680/

In [62]:
def data_to_bins(data):
    counts=np.zeros(100)
    for dat in data:
        try:
            counts[int(dat*100)]+=1/len(data)
        except IndexError:
            pass
    return counts

In [70]:
samples_new = []
for hist, par in samples_load:
    samples_new.append((data_to_bins(hist), par))

In [73]:
import pickle
with open('ML_proj_samps.dat', 'wb') as file:
    pickle.dump(samples_new, file)

In [8]:
import pickle
with open('ML_proj_samps.dat', 'rb') as file:
    samples_load = pickle.load(file)

In [74]:
samples_train = samples_new[:8000]
samples_val = samples_new[8000:]

In [44]:
hist, params = samples_load[0]
hist.astype('double')

array([0.24111, 0.20476, 0.2126 , ..., 0.24117, 0.20626, 0.24237])

In [60]:
class proj_dataset(Dataset):
    
    def __init__(self,samples):
        super(Dataset, self).__init__()
        self.samples = samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        hist, params = self.samples[idx]
        hist = torch.from_numpy(hist).float()
        params = torch.Tensor(params).float()
        return hist, params

In [102]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.input_layer = nn.Linear(100, 50)
        self.output_layer = nn.Linear(20, 4)
        self.hidden_layers = nn.Sequential(
            nn.Linear(50, 100),
            nn.Sigmoid(),
            nn.Dropout(p=0.5),
            nn.Linear(100,25),
            nn.Sigmoid(),
            nn.Linear(25,25),
            nn.Sigmoid(),
            nn.Linear(25,20),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        return x   
        

In [82]:
#Trainig & validation loop
def fit(model, samples_train, samples_val, batch_size=10, lr=0.05):    
    from torchmetrics import MeanSquaredError
    data_loader_train = DataLoader(proj_dataset(samples_train), shuffle=True, batch_size=batch_size)
    data_loader_val = DataLoader(proj_dataset(samples_val), shuffle=True, batch_size=batch_size)
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr) 
    metrics = MeanSquaredError()

    for epoch in range(10):
        #Training
        model.train()
        for hist, params in data_loader_train:
            optimizer.zero_grad()
            pred = model(hist)
            loss = criterion(pred, params)
            loss.backward()
            optimizer.step()
            metrics(pred, params)
        train_loss = metrics.compute()
        metrics.reset()
        #Validation
        model.eval()
        with torch.no_grad():
            for hist, params in data_loader_val:
                pred = model(hist)
                metrics(pred, params)
        val_loss = metrics.compute()
        metrics.reset()
        
        print(f'Epoch {epoch+1}/10: training loss {train_loss}, validation loss {val_loss}')
    
    return model    
    
             

In [83]:
# Evaluation

def evaluate_model(model, samples_eval, batch_size=10):
    import torchmetrics
    metrics = torchmetrics.MeanSquaredError()
    data_loader = DataLoader(proj_dataset(samples_eval), shuffle=True, batch_size=batch_size)
    model.eval()
    with torch.no_grad():
        for hist, params in data_loader:
            pred = model(hist)
            metrics(pred, params)
    return metrics.compute()    


In [103]:
model = fit(Net(), samples_train, samples_val)


Epoch 1/10: training loss 0.031431976705789566, validation loss 0.009541042149066925
Epoch 2/10: training loss 0.009355119429528713, validation loss 0.008624564856290817
Epoch 3/10: training loss 0.00927263218909502, validation loss 0.01006998959928751
Epoch 4/10: training loss 0.009325104765594006, validation loss 0.008918026462197304
Epoch 5/10: training loss 0.009328906424343586, validation loss 0.008558460511267185
Epoch 6/10: training loss 0.009346254169940948, validation loss 0.009577351622283459
Epoch 7/10: training loss 0.00935167446732521, validation loss 0.008997203782200813
Epoch 8/10: training loss 0.00934012420475483, validation loss 0.009078686125576496
Epoch 9/10: training loss 0.009336731396615505, validation loss 0.009032435715198517
Epoch 10/10: training loss 0.00928193423897028, validation loss 0.008775408379733562


In [105]:
torch.save(model.state_dict(), 'proj_model.model')

In [104]:
model.eval()
model(torch.from_numpy(samples_new[10][0]).float())

tensor([ 0.2827, -0.0024,  0.2314,  0.2634], grad_fn=<ViewBackward0>)

In [97]:
samples_new[10][1]

[0.30940762059894866,
 -0.005726067205798068,
 0.27777555818594596,
 0.13504126302915936]

In [ ]:
model_loaded = Net()
model_loaded.load_state_dict(torch.load('proj_model.model'))
model_loaded.eval()

## Test with Werner states

In [92]:
df = pd.read_csv('werner_sample.csv', index_col='Unnamed: 0')
df_test = pd.read_csv('werner_test.csv', index_col='Unnamed: 0')
df.head()

,Angle,Visibility,"[0.0, 0.01]","[0.01, 0.02]","[0.02, 0.03]","[0.03, 0.04]","[0.04, 0.05]","[0.05, 0.06]","[0.06, 0.07]","[0.07, 0.08]",...,"[0.9, 0.91]","[0.91, 0.92]","[0.92, 0.93]","[0.93, 0.94]","[0.94, 0.95]","[0.95, 0.96]","[0.96, 0.97]","[0.97, 0.98]","[0.98, 0.99]","[0.99, 1.0]"
0,0.333026,0.045424,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.568860,0.448408,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.620751,0.267946,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.127500,0.635826,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.401182,0.933977,0.0,0.00898,0.02606,0.02698,0.02618,0.02702,0.02636,0.0261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
class werner_dataset(Dataset):
    
    def __init__(self,samples):
        super(Dataset, self).__init__()
        self.samples = samples
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        hist, params = self.samples[idx]
        hist = torch.from_numpy(hist).float()
        params = torch.Tensor(params).float()
        return hist, params